In [1]:
import json 

with open("./save/dataset.json", encoding="utf-8") as f:
    data = json.load(f)

In [2]:
import joblib
from scipy import sparse 

vec = joblib.load('./save/tfidf/vectorizer.joblib')
X = sparse.load_npz('./save/tfidf/X.npz')

In [3]:
print(X.shape)

(100000, 2650092)


In [4]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(
    n_components=100,
    n_iter=10,
    random_state=42
)

Xk = svd.fit_transform(X)

In [10]:
N, k = Xk.shape 
print(N, k)

100000 100


There are two options for navigable small world searches: greedy and beam. 

Greedy means pick a random node, move to the neighboring node with closest distance to query, terminate when current node is better than neighbors.

Beam means pick a random node, push all `M` neighboring nodes to a `max_len=ef_query` heap, and explore the next best node. Terminate when the worst of the `k` nearest to query is better than the best of the `ef_query` within heap.    


In [12]:
import hnswlib
import numpy as np 

index = hnswlib.Index(space='cosine', dim=k)
index.init_index(
    max_elements=N,
    ef_construction=200, # when adding a node, size of minheap to determine best neighbors so far 
    M=32 # connections each document has in a navigable small world
)

ids = np.arange(N)
index.add_items(Xk, ids)

index.set_ef(50)

In [52]:
def search_hnsw_svd(doc_ind: int, k: int = 20):
    q = Xk[doc_ind]
    labels, distances = index.knn_query(q, k=k+1) 
    labels, distances = labels[0], distances[0]
    
    print(f"Query: {data[doc_ind]['title']}\n")
    for idx, distance in zip(labels[1:], distances[1:]):
        print(f"{data[idx]['title']} {distance}")
    return [(distance, data[idx]['title']) for idx, distance in zip(labels[1:], distances[1:])]

Cosine similarity is low between wikipedia articles because `vocab_size=2650092`. This is probably too large and so relationships are weak. Furthermore, common words probably occur too often and so the inverse document frequency weight is making values way too low.

Anyway, this repository is meant more so to understand how to do nearest neighbor and not worry about whether the search actually means something. I think the search is good enough for a proof of concept. 

In [58]:
search_hnsw_svd(51231, 5)

Query: River HomeLink

Joseph Burr Tyrrell Elementary School 0.0537259578704834
Tulsa School of Arts and Sciences 0.07285439968109131
Juja 0.07441812753677368
Loreto High School, Limuru 0.08009469509124756
Rocky Hill High School 0.08073699474334717


[(0.053725958, 'Joseph Burr Tyrrell Elementary School'),
 (0.0728544, 'Tulsa School of Arts and Sciences'),
 (0.07441813, 'Juja'),
 (0.080094695, 'Loreto High School, Limuru'),
 (0.080736995, 'Rocky Hill High School')]